In [9]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [10]:
# import all your modules here
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [11]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [12]:
print(baseline)
print(study)

['2023-04-01', '2023-06-30']
['2023-07-01', '2023-08-01']


In [13]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)

In [14]:
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [15]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [16]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': baseline[0], 'end': study[1]}

df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Baseline:{baseline}, Study:{study}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    

    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )

    current_idx = idx_end+1
    
    if (current_idx > n_vars):
        current_idx = n_vars

    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")

    if response.status_code != 204:

        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)


Making request for the following interval: Baseline:['2023-04-01', '2023-06-30'], Study:['2023-07-01', '2023-08-01']
Progress: 3.1%
Progress: 5.9%
Progress: 8.8%
Progress: 11.6%
Progress: 14.4%
Progress: 17.2%
Progress: 20.1%
Progress: 22.9%
Progress: 25.7%
Progress: 28.5%
Progress: 31.4%
Progress: 34.2%
Progress: 37.0%
Progress: 39.8%
Progress: 42.7%
Progress: 45.5%
Progress: 48.3%
Progress: 51.1%
Progress: 54.0%
Progress: 56.8%
Progress: 59.6%
Progress: 62.4%
Progress: 65.3%
Progress: 68.1%
Progress: 70.9%
Progress: 73.7%
Progress: 76.6%
Progress: 79.4%
Progress: 82.2%
Progress: 85.0%
Progress: 87.9%
Progress: 90.7%
Progress: 93.5%
Progress: 96.3%
Progress: 99.2%
Progress: 100.0%


In [17]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)